### functionality

### required directory structure

### input initialization

In [ ]:
subdirName = 'M_DD_YYYY' # Name of "Observational Data" subdirectory to be created, usually of the format M_DD_YYYY

refants = ['DA59','DV11','DA49','DV12','PM01','DV07'] # Can be any number of antenna names, though plots generally
                                                      # require six. Order doesn't matter

gendir = '/.../Offset Drift/Observational Data/' # "Observational Data" location, assuming specified structure

In [6]:
subdirName = '8_9_2021' # Name of "Observational Data" subdirectory to be created, usually of the format M_DD_YYYY.

refants = ['DA59','DV11','DA49','DV12','PM01','DV07'] # Can be any number of antenna names, though plots generally
                                                      # require six. Order doesn't matter.

gendir = '/Users/colemeyer/Documents/Offset Drift/Observational Data/' # "Observational Data" location, assuming specified structure.

***

### imports

In [7]:
import os
#import re
#import numpy as np
#import pandas as pd
from Packages import Miscellaneous as mc
#from Packages import TimeConvert as tc

ModuleNotFoundError: No module named 'Packages'

# Data Extraction from ASDMs in "Observational Data" subdirectory

In [5]:
# May need to be updated eventually. Must include all weather stations that appear in Weather.xml
weastats = ['Meteo129','Meteo130','Meteo131','Meteo201','Meteo309','Meteo410','MeteoCentral','MeteoItinerant']

gendir = gendir+subdirName+'/ASDMs/'

antArrHeaders = ['antname','padname','statnum','xPhaseAvg','yPhaseAvg','poserrX','poserrY','poserrZ','poserr()X','poserr()Y','poserr()Z','newposX','newposY','newposZ','geoerrX','geoerrY','geoerrZ','geoposX','geoposY','geoposZ','delayrms']
weaArrHeaders = ['padname','statnum','geoposX','geoposY','geoposZ','pressure','relHum','temp','windDir','windSpeed','windMax']

numObs = mc.numObs(gendir)

for ASDM in os.listdir(gendir):
    if "uid" in ASDM:
        asdmdir = gendir+ASDM+'/'

        # Open all input files from offline solution
        antXML = open(asdmdir+'Antenna.xml',"r")
        statXML = open(asdmdir+'Station.xml',"r")
        weaXML = open(asdmdir+'Weather.xml',"r")
        phaseXML = open(asdmdir+'CalPhase.xml',"r")
        for file in os.listdir(asdmdir):
            if ".log" in file:
                log = open(asdmdir+file,"r")
        '''
        # Create all pre-CSV arrays
        antArr = np.empty((1,21),dtype='U25')
        weaArr = np.empty((1,11),dtype='U25')

        # Prepare all input files for looping
        ##########
        line = antXML.readline()
        while "<ContainerEntity" not in line:
            line = antXML.readline()
        ##########
        line = statXML.readline()
        while "<ContainerEntity" not in line:
            line = statXML.readline()
        ##########
        line = weaXML.readline()
        while "<ContainerEntity" not in line:
            line = weaXML.readline()
        ##########
        line = phaseXML.readline()
        while "<ContainerEntity" not in line:
            line = phaseXML.readline()
        ##########
        line = log.readline()
        while "Ref." not in line:
            line = log.readline()
        ##########


        ########## Antenna.xml Cycling
        line = antXML.readline()
        while "</AntennaTable>" not in line:
            antXML.readline()

            antRow = np.empty(21,dtype='U25')
            antRow[0] = re.sub(r"<.*?>", "", antXML.readline()).strip() # antname
            for i in range(6): line = antXML.readline()
            antRow[2] = re.sub(r"<.*?>", "", antXML.readline()).strip() # statnum
            for i in range(2): line = antXML.readline()

            antArr = np.vstack((antArr,antRow)) # Add row to antArr

        antArr = antArr[1:,:] # Remove first row (consequence of row-adding method)


        ########## Station.xml Cycling
        row = 0
        line = statXML.readline()
        while "</StationTable>" not in line:
            if row < np.shape(antArr)[0]:
                statXML.readline()

                antArr[row,1] = re.sub(r"<.*?>", "", statXML.readline()).strip() # padname
                line = statXML.readline().split()
                for i in range(2,4): antArr[row,i+15] = line[i] # geopos (X,Y)
                antArr[row,19] = line[4].rstrip("</position>\n") # geopos (Z)
                for i in range(3): line = statXML.readline()
            else:
                weaRow = np.empty(11,dtype='U25')
                weaRow[1] = re.sub(r"<.*?>", "", statXML.readline()).strip() # statnum
                weaRow[0] = re.sub(r"<.*?>", "", statXML.readline()).strip() # padname
                line = statXML.readline().split()
                for i in range(2,4): weaRow[i] = line[i] # geopos (X,Y)
                weaRow[4] = line[4].rstrip("</position>\n") # geopos (Z)
                for i in range(3): line = statXML.readline()

                weaArr = np.vstack((weaArr,weaRow)) # Add row to weaArr

            row += 1

        weaArr = weaArr[1:,:] # Remove first row (consequence of row-adding method)


        ########## Weather.xml Cycling
        tempWeaArr = np.zeros((len(weaants),7),dtype='float')

        line = weaXML.readline()
        while "</WeatherTable>" not in line:
            line = weaXML.readline()

            temp = np.empty(6,dtype='float')
            for i in range(6): temp[i] = re.sub(r"<.*?>", "", weaXML.readline()).strip() # All weather attributes
            statNum = weaXML.readline().lstrip("    <stationId>").rstrip("</stationId>\n")
            for i in range(2): line = weaXML.readline()

            
            The strategy below is a bit tricky. Since it's tricky adding rows to 3D numpy arrays, I've decided to
            implement a sort of working average concept. The idea is that tempWeaArr has len(weaants) rows, each 
            with 7 columns. These 7 columns are the 6 weather attributes plus the "number of rows" so that we can 
            take the average correctly. For each chunk of weather data for a given station in Weather.xml, the 
            working average is updated along with the "number of rows".
            

            for i in range(len(weaants)):
                if weaArr[i,1] == statNum and temp[0] != 0:
                    tempWeaArr[i,:6] = tempWeaArr[i,:6] + temp # Summation portion of "working average"
                    tempWeaArr[i,6] += 1

        for row in range(np.shape(tempWeaArr)[0]):
            if tempWeaArr[row,6] != 0:
                tempWeaArr[row,:6] = tempWeaArr[row,:6] / tempWeaArr[row,6] # Division portion of "working average"

        for i in range(len(weaants)):
            for j in range(6):
                weaArr[i,5:] = tempWeaArr[i,:6]

        ########## CalPhase.xml Cycling
        line = phaseXML.readline()
        while "<singleAntennaName>" not in line: line = phaseXML.readline()
        line = re.sub(r"<.*?>", "", line).strip().split()

        antNames = []
        for i in range(2,int(line[1])+2): antNames.append(line[i][1:-1])

        phaseXML = open(uidDir+'CalPhase.xml',"r")
        phaseData = np.zeros((len(antNames),3),dtype='float')

        for i in range(5): line = phaseXML.readline()
        while "</CalPhaseTable>" not in line:
            for i in range(108): phaseXML.readline() # Skips BB_1, BB_2, BB_3, and BB_4 entirely
            for i in range(21): phaseXML.readline() # Skips to phaseAnt data within BB_ALL

            line = re.sub(r"<.*?>", "", phaseXML.readline()).strip().split()

            
            I've decided to adopt the same "working average" concept above here, since the data comes in a
            similar format.
            

            # Processing <phaseAnt> data within BB_ALL
            temp = line[2]
            line = line[3:]
            for j in range(2):
                for i in range(int(j*len(line)/2),int((j+1)*len(line)/2)):
                    if line[i] != 0:
                        phaseData[int(i%(len(line)/2)),j] = phaseData[int(i%(len(line)/2)),j] + float(line[i]) # Summation portion of "working average"
            phaseData[:,2] += 1

            for i in range(5): line = phaseXML.readline()

        for i in range(2): phaseData[:,i] = phaseData[:,i] / phaseData[:,2] # Division portion of "working average"
        phaseData = phaseData[:,:2]

        antArr[:,3:5] = phaseData[:,:]


        ########## .Log Cycling
        row = 1
        line = log.readline()
        while row < np.shape(antArr[:,0])[0]:
            line = line.split()
            for i in range(3): antArr[row,i+5] = line[2*(i+2)] # poserr (X,Y,Z)
            for i in range(2): antArr[row,i+8] = line[2*(i+3)-1][1:-3] # poserr() (X,Y)
            antArr[row,10] = line[9][1:-2] # poserr() (Z)
            antArr[row,20] = line[12]
            line = log.readline().split()
            for i in range(3): antArr[row,i+11] = line[2*(i+1)] # newpos (X,Y,Z)
            line = log.readline().split()
            for i in range(3): antArr[row,i+14] = line[2*(i+1)] # geoerr (X,Y,Z)
            for i in range(2): line = log.readline()

            row += 1

        weaArr[weaArr == "0.0"] = np.nan
        antArr[antArr == "0.0"] = np.nan
        antArr[antArr == ""] = np.nan
        pd.DataFrame(weaArr).to_csv(uidDir+'WeaData.csv', index=None, header=weaArrHeaders) # Export weaArr to WeaData.csv
        pd.DataFrame(antArr).to_csv(uidDir+'AntData.csv', index=None, header=antArrHeaders) # Export antArr to AntData.csv
'''

'\nnumObs = mc.numObs(genDir)\n\nfor uid in os.listdir(genDir):\n    if uid[:3] == "uid":\n        uidDir = genDir+uid+\'/\'\n\n        # Open all input files from offline solution\n        antXML = open(uidDir+\'Antenna.xml\',"r")\n        statXML = open(uidDir+\'Station.xml\',"r")\n        weaXML = open(uidDir+\'Weather.xml\',"r")\n        phaseXML = open(uidDir+\'CalPhase.xml\',"r")\n        for file in os.listdir(uidDir):\n            if ".log" in file:\n                log = open(uidDir+file,"r")\n\n        # Create all pre-CSV arrays\n        antArr = np.empty((1,21),dtype=\'U25\')\n        weaArr = np.empty((1,11),dtype=\'U25\')\n\n        # Prepare all input files for looping\n        ##########\n        line = antXML.readline()\n        while "<ContainerEntity" not in line:\n            line = antXML.readline()\n        ##########\n        line = statXML.readline()\n        while "<ContainerEntity" not in line:\n            line = statXML.readline()\n        ##########\n      

# observation combination

In [15]:
antArr = np.empty((len(refants),numObs,23),dtype='U25')
weaArr = np.empty((len(weaants),numObs,13),dtype='U25')

row = 0
for uid in os.listdir(genDir):
    if uid[:3] == "uid":
        uidDir = genDir+uid+'/'
        
        # Load in antenna and weather station data
        antData = pd.read_csv(uidDir+'/AntData.csv')
        weaData = pd.read_csv(uidDir+'/WeaData.csv')
        antData.index = antData.antname
        weaData.index = weaData.padname
        
        # Extract time from ASDM.xml
        asdmXML = open(uidDir+'/ASDM.xml',"r")
        for i in range(4): line = asdmXML.readline()
        time = tc.FormattedTimetoUnix(re.sub(r"<.*?>", "", line).strip()) # time

        # Combine time, uid, and AntData.csv into one place
        for ant in range(len(refants)):
            antArr[ant,row,:2] = np.array([time,uid],dtype='U25')
            if refants[ant] in antData.index:
                antArr[ant,row,2:] = antData.loc[refants[ant]].to_numpy()
            else: antArr[ant,row,2:] = np.nan

        # Combine time, uid, and WeaData.csv into one place
        for stat in range(len(weaants)):
            weaArr[stat,row,:2] = np.array([time,uid],dtype='U25')
            weaArr[stat,row,2:] = weaData.loc[weaants[stat]].to_numpy()

        row += 1
    
# Create new "offsetdata" and "weatherdata" directories, if they don't already exist
if not os.path.isdir(directory+'/'+date+'/offsetdata'):
    os.mkdir(directory+'/'+date+'/offsetdata')
if not os.path.isdir(directory+'/'+date+'/weatherdata'):
    os.mkdir(directory+'/'+date+'/weatherdata')

# Export to "offsetdata"
for i in range(len(refants)): 
    temp = pd.DataFrame(antArr[i,(antArr[i,:,0].argsort())],columns=['time','uid','antname','padname','statnum','xPhaseAvg','yPhaseAvg','poserrX','poserrY','poserrZ','poserr()X','poserr()Y','poserr()Z','newposX','newposY','newposZ','geoerrX','geoerrY','geoerrZ','geoposX','geoposY','geoposZ','delayrms'])
    temp = temp.drop(columns='antname')
    pd.DataFrame(temp).to_csv(directory+'/'+date+'/offsetdata/'+refants[i]+'.csv', index=None)

# Export to "weatherdata"
for i in range(len(weaants)): 
    temp = pd.DataFrame(weaArr[i,(weaArr[i,:,0].argsort())],columns=['time','uid','padname','statnum','geoposX','geoposY','geoposZ','pressure','relHum','temp','windDir','windSpeed','windMax'])
    temp = temp.drop(columns='padname')
    pd.DataFrame(temp).to_csv(directory+'/'+date+'/weatherdata/'+weaants[i]+'.csv', index=None)
    
print("done")

done
